**Here I will try to implement ResNet 50 on the 102 Flowers dataset. I begin with loading the data. Then  I train and evaluate the performance of ResNet 50 on the loaded data.**

In [ ]:
# Downloading the main data files
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!tar -xzf 102flowers.tgz
!rm 102flowers.tgz
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat

In [ ]:
# Importing Needed Libraries
import tarfile as tf
import os
import numpy as np
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from scipy.io import loadmat
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split as tts
from keras.utils import to_categorical
from collections import Counter

print ('Import Completed')

def set_device(): # Function to help with determining the accelerator
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  if device == 'cuda':
    print('GPU yay!')
  elif device == 'cpu':
    print('CPU :|')
  return device
DEVICE = set_device()

In [ ]:
# Load the images from the folder into a python list, as numpy arrays
image_folder = "/kaggle/working/jpg"
all_imgs = list()
all_labels = []
for img_name in tqdm(os.listdir(image_folder)):
    image_path = os.path.join(image_folder, img_name)
    img = Image.open(image_path)
    img_array = np.array(img)
    all_imgs.append(img_array)
    all_labels.append(int(img_name[6:11])-1)
    # see whether there are any images with different shapes or dimensions in the dataset
    if len(img_array.shape) != 3 or img_array.shape[2] != 3:
        print('Differing dimension/shape detected!')

print(f"Number of total images is: {len(all_imgs)}")

In [ ]:
# Load the Labels as Mat objects and explore them
img_labels = loadmat("imagelabels.mat")
print(img_labels.keys())
img_labels = img_labels['labels'][0]
print(f"Number of total labels is: {len(img_labels)}")

# convert the indices in all_labels list to the actual label of their corresponding image
all_labels = [img_labels[i] - 1 for i in all_labels] # Since the to_categorical function expects values to start from 0,
print(f"Range of labels: {min(all_labels)} - {max(all_labels)}") # I subtract 1 from all label values.

# convert numbers to
all_labels = to_categorical(all_labels, num_classes = 102, dtype = float)

print(f"Number of classes: {all_labels.shape[1]}")

# To make sure there are enough examples for each of the labels, Let's check the min and max occurences.
label_counts = Counter(img_labels)
print(f"Most repeated label is {label_counts.most_common()[0][0]}: {label_counts.most_common()[0][1]} times")
print(f"Least repeated label is {label_counts.most_common()[-1][0]}: {label_counts.most_common()[-1][1]} times")

In [ ]:
# And now let's visualize the distribution of frequencies:
values = label_counts.keys()
counts = label_counts.values()
plt.bar(values, counts)
# Generate a uniform distribution to visually compare with our data
num_samples = len(img_labels)
min_value = min(img_labels)
max_value = max(img_labels)
uniform_data = np.random.randint(min_value, max_value + 1, num_samples) # create a uniform distribution within the same range

# Count the occurrences of each value in the uniform distribution
uniform_value_counts = Counter(uniform_data)

# Extract values and counts for plotting
uniform_values = list(uniform_value_counts.keys())
uniform_counts = list(uniform_value_counts.values())
# Create the bar chart
plt.bar(uniform_values, uniform_counts, alpha=0.25, color='red', label='Uniform Distribution')

plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Flowers')
plt.show()

In [ ]:
# Here I split the data into train, validation, and test sets.
trainx, testx, trainy, testy = tts(all_imgs, all_labels, test_size = 0.2, random_state = 7)
valx, testx, valy, testy = tts(testx, testy, test_size = 0.5, random_state = 7)
# Create datasets corresponding to train, valid, and test data, to load them later on via dataloaders to the neural network.
class data_set(Dataset):
    def __init__(self, img, label, transform = None):
        self.img = img
        self.label = label
        self.transform = transform
    def __getitem__(self, idx):
        img = self.img[idx]
        label = self.label[idx]
        if self.transform:
            img = self.transform(img)
        return img, label
    def __len__(self):
        return len(self.label)
    
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((112, 112), interpolation=Image.BILINEAR, antialias = None)
])

batch_size = 128 # This batch size seems to work well with my resources!
train_set = data_set(trainx, trainy, transform = transform)
val_set = data_set(valx, valy, transform = transform)
test_set = data_set(testx, testy, transform = transform)

train_loader = DataLoader(train_set, batch_size, shuffle = True)
val_loader = DataLoader(val_set, batch_size, shuffle = False)
test_loader = DataLoader(test_set, batch_size, shuffle = False)

In [ ]:
# Now Let's load the ResNet 50 and intiialize its hyperparamteres.

resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) # set the model to be loaded as pretrained
for param in resnet.parameters(): # I freeze the weights to only train the classifier layer (here it is frozen),
    param.requires_grad = False     # while keeping the model's feature extraction capability


# And next I modify the final later to output the same number of categories in the dataset.
resnet.fc = nn.Sequential(
            nn.Linear(resnet.fc.in_features, 102),
            nn.Softmax(dim = 1)  # Add a softmax activation to produce class probabilities.
            )

resnet.to(DEVICE) # Hopefully Cuda!
print('Resnet 50 Loaded!')

# And here I set the loss function to cross entropy loss since I want to multi-class classification
loss_f = nn.CrossEntropyLoss()
epochs = 25

In [ ]:
# Now I can train the pretrained Model on train set.
# Also I assign some variables to measure the model's performance while training.
train_losses = list()
train_accuracies = list()
val_losses = list()
val_accuracies = list()

for epoch in tqdm(range(epochs)): 
    lr = 2e-3 * 0.99 ** (epoch-1) # use learning rate decay
    optimizer = optim.Adam(resnet.parameters(), lr = lr) # Use Adam optimizer for faster gradient decent steps
    resnet.train() # training mode on

    running_loss = 0.0
    num_samples = 0
    num_correct = 0
    for img, label in train_loader:
        img, label = img.to(DEVICE), label.to(DEVICE) # move all variable to GPU before feeding to network
        optimizer.zero_grad() # do the training loop and update model parameters (FC layer only)
        output = resnet(img)
        loss = loss_f(output, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output, 1)  # Get the class with the highest probability
        _, true_label = torch.max(label, 1)
        num_samples += label.size(0)
        num_correct += (predicted == true_label).sum().item()
        
    train_accuracy = 100 * num_correct / num_samples
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    train_accuracies.append(train_accuracy)
    
    with torch.no_grad():
        vr_loss = 0
        val_samples = 0
        val_corrects = 0
        for img, label in val_loader:
            img, label = img.to(DEVICE), label.to(DEVICE)
            output = resnet(img)
            _, predicted = torch.max(output, 1)
            _, true_label = torch.max(label, 1)
            
            val_samples += label.size(0)
            val_corrects += (predicted == true_label).sum().item()
            vr_loss += loss_f(output, label).item()
        val_accuracy = 100 * val_corrects/val_samples
        val_loss = vr_loss/len(val_loader)
        print(f"Epoch [{epoch + 1}/{epochs}] - Training Loss: {epoch_loss:.4f} - Training Accuracy: {train_accuracy:.2f}%")
        print(f"Epoch [{epoch + 1}/{epochs}] - Validation Loss: {val_loss:.4f} - Validation Accuracy: {val_accuracy:.2f}%")
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

In [ ]:
# Now the final step: test the model
num_sample = 0
num_correct = 0
preds = []
true_labels = []
with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to(DEVICE), label.to(DEVICE)
        output = resnet(img) 
        _, predicted = torch.max(output, 1)
        _, true_label = torch.max(label, 1)
        preds += predicted.cpu().numpy().tolist()
        true_labels += true_label.cpu().numpy().tolist()
        num_sample += label.size(0)
        num_correct += (predicted == true_label).sum().item()
    test_accuracy = 100 * num_correct/num_sample
    print(f"Test Accuracy: {test_accuracy:.2f}%")


In [ ]:
# And visualize the performance
# plot train loss by epoch
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss', color = 'blue')
plt.plot(val_losses, label='Validation Loss', color = 'red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('Loss_plot.png')

# plot training and test accuracy curves
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy', color = 'blue')
plt.plot(val_accuracies, label='Validation Accuracy', color = 'red')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.axhline(y=test_accuracy, color='red', linestyle='--', label='Test Accuracy')
plt.legend()
plt.savefig('Acc_plot.png')

plt.show()

In [ ]:
pred_count = Counter(preds)
true_count = Counter(true_labels)
# And now let's visualize the distribution of frequencies:
p_values = pred_count.keys()
p_counts = pred_count.values()
plt.bar(p_values, p_counts, label = 'Predicted Labels')

t_values = list(true_count.keys())
t_counts = list(true_count.values())
# Create the bar chart
plt.bar(t_values, t_counts, alpha=0.25, color='red', label='True Labels')

plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.title('Frequency of Predictions vs True Labels')
plt.legend()
plt.savefig('pred_vs_true_plot.png')
plt.show()